![Nuclio logo](https://nuclio.school/wp-content/uploads/2018/12/nucleoDS-newBlack.png)

# TFM - Impacto de la alimentación sobre la salud de las personas

## Tabla de Contenidos

* [A. Introduccion](#introduction)
* [B. Importar Librerias](#libraries)
* [C. Importar datos](#data)

## A. Introducción <a class="anchor" id="introduction"></a>

## B. Importar Librerías<a class="anchor" id="libraries"></a>

In [14]:
import os
import sys
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt


import seaborn as sns
from sklearn import preprocessing

import pickle


## C. Importar datos<a class="anchor" id="data"></a>

In [15]:
PATH_COMUNIDADES = 'data/comunidades/'

In [66]:
def LeerDatosComunidades(path_comunidades):
    # Leemos los nombres de los ficheros
    paths_comunidades = [path for path in os.listdir(path_comunidades) if path.endswith('.xlsx')]

    # Creamos una lista para ir añadiendo los dataframes leidos
    dfs = list()

    for path_comunidad in paths_comunidades:
        # Extraemos el nombre de la comunidad
        nombre_comunidad = os.path.splitext(path_comunidad)[0]

        # Leemos el fichero, reseteamos el index y le cambiamos el nombre de la columna a ID
        fichero = pd.read_excel('./data/comunidades/'+path_comunidad).reset_index(drop=True).rename(columns={'Unnamed: 0': 'ID'})

        # Descartamos las filas donde el la columna BIO no es nula y asignamos el nombre de la comunidad a una nueva columna.
        fichero_sin_bio = fichero[fichero['SEGMENTO BIO'].isna()].reset_index(drop=True).assign(COMUNIDAD=nombre_comunidad)

        # Añadimos el fichero leido a nuestra lista de dataframes
        dfs.append(fichero_sin_bio)

    # Montamos el dataframe final
    datos = pd.concat(dfs, ignore_index=True)

    # Eliminamos la columna SEGMENTO BIO
    datos.drop('SEGMENTO BIO', axis=1, inplace=True)

    # Cambiamos el orden de la columna comunidad
    cols = datos.columns.tolist()
    cols = cols[:5] + cols[-1:] + cols[5:-1]
    datos = datos[cols]

    return datos

In [67]:
df_comunidades = LeerDatosComunidades(PATH_COMUNIDADES)

In [68]:
df_comunidades.head(20)

,ID,SECTOR,SECCION,CATEGORIA,FAMILIA,COMUNIDAD,SEMANA 40-16,SEMANA 41-16,SEMANA 42-16,SEMANA 43-16,...,SEMANA 34-19,SEMANA 35-19,SEMANA 36-19,SEMANA 37-19,SEMANA 38-19,SEMANA 39-19,SEMANA 40-19,SEMANA 41-19,SEMANA 42-19,SEMANA 43-19
0,TOTAL PGC,NaN,NaN,NaN,NaN,Ceuta_y_Melilla,0.0,0.0,0.0,0.0,...,2652.71,2853.52,2699.68,2703.08,2484.19,2853.67,2844.06,2412.15,2559.18,2647.53
1,ALIM. Y BEBIDAS,ALIM. Y BEBIDAS,NaN,NaN,NaN,Ceuta_y_Melilla,0.0,0.0,0.0,0.0,...,1328.36,1421.87,1364.16,1405.32,1269.16,1457.10,1477.37,1288.87,1375.99,1442.49
2,ALIM.SECA,ALIM. Y BEBIDAS,ALIM.SECA,NaN,NaN,Ceuta_y_Melilla,0.0,0.0,0.0,0.0,...,704.00,782.74,760.74,791.09,717.49,829.03,829.07,739.01,813.17,891.25
3,ACEITE,ALIM. Y BEBIDAS,ALIM.SECA,ACEITE,NaN,Ceuta_y_Melilla,0.0,0.0,0.0,0.0,...,60.80,72.97,67.83,71.55,62.32,73.07,76.68,61.45,60.28,63.69
4,GIRASOL..,ALIM. Y BEBIDAS,ALIM.SECA,ACEITE,GIRASOL..,Ceuta_y_Melilla,0.0,0.0,0.0,0.0,...,18.18,20.78,20.41,20.16,18.68,21.06,22.47,16.94,17.49,19.44
5,OLIVA,ALIM. Y BEBIDAS,ALIM.SECA,ACEITE,OLIVA,Ceuta_y_Melilla,0.0,0.0,0.0,0.0,...,39.07,48.58,44.35,47.56,40.15,47.73,50.83,41.63,39.63,41.40
6,RESTO ACEITES,ALIM. Y BEBIDAS,ALIM.SECA,ACEITE,RESTO ACEITES,Ceuta_y_Melilla,0.0,0.0,0.0,0.0,...,3.54,3.61,3.06,3.84,3.49,4.29,3.39,2.88,3.17,2.85
7,ADITIVOS COCINA,ALIM. Y BEBIDAS,ALIM.SECA,ADITIVOS COCINA,NaN,Ceuta_y_Melilla,0.0,0.0,0.0,0.0,...,13.87,15.52,14.98,16.48,15.77,16.12,16.09,14.38,14.43,17.00
8,ADEREZOS,ALIM. Y BEBIDAS,ALIM.SECA,ADITIVOS COCINA,ADEREZOS,Ceuta_y_Melilla,0.0,0.0,0.0,0.0,...,0.28,0.25,0.25,0.27,0.22,0.31,0.24,0.19,0.18,0.21
9,ESPECIAS,ALIM. Y BEBIDAS,ALIM.SECA,ADITIVOS COCINA,ESPECIAS,Ceuta_y_Melilla,0.0,0.0,0.0,0.0,...,5.53,6.47,6.14,6.45,6.15,6.57,6.62,6.22,6.01,8.11


In [69]:
lista_sectores = list(df_comunidades[~df_comunidades['SECTOR'].isna()]['SECTOR'].unique())
lista_secciones = list(df_comunidades[~df_comunidades['SECCION'].isna()]['SECCION'].unique())
lista_categorias = list(df_comunidades[~df_comunidades['CATEGORIA'].isna()]['CATEGORIA'].unique())
lista_familias = list(df_comunidades[~df_comunidades['FAMILIA'].isna()]['FAMILIA'].unique())

In [73]:
len(lista_sectores), len(lista_secciones), len(lista_categorias), len(lista_familias)

(3, 11, 95, 414)

In [75]:
df_comunidades[df_comunidades['SECTOR'] == lista_sectores[1]]['SECCION'].unique()

array([nan, 'DROGUERIA Y LIMPIEZA', 'PERFUMERIA E HIGIENE'], dtype=object)

In [141]:
diccionario_esquema = {}

for sector in lista_sectores:
    categorias_sector = pd.Series(df_comunidades[df_comunidades['SECTOR'] == sector]['SECCION'].unique())
    categorias_sector = categorias_sector[categorias_sector.notna()]
    for categoria in categorias_sector:
        #print(sector + ' : ' + categoria)
        diccionario_esquema[sector] = {
            diccionario_sector[categoria] : ''
        }

KeyError: 'ALIM.SECA'

In [138]:
diccionario_esquema

{'ALIM. Y BEBIDAS': {'LECHE Y BATIDOS'},
 'DROGUERIA Y PERFUMERIA': {'PERFUMERIA E HIGIENE'},
 'PROD.FRESCOS': {'QUESOS'}}

In [122]:
categorias_sector = pd.Series(df_comunidades[df_comunidades['SECTOR'] == lista_sectores[0]]['SECCION'].unique())
categorias_sector[categorias_sector.notna()].to_dict()

{1: 'ALIM.SECA', 2: 'BEBIDAS', 3: 'CONSERVAS', 4: 'LECHE Y BATIDOS'}